In [ ]:
# Implementación de Reglas de Asociación Multilevel
import pandas as pd
import numpy as np
from itertools import combinations

# Definir jerarquía de productos
jerarquia = {
    # Nivel 3 (específico) -> Nivel 2 (subcategoría) -> Nivel 1 (categoría)
    'Leche_Entera': {'nivel2': 'Lácteos', 'nivel1': 'Alimentos'},
    'Leche_Descremada': {'nivel2': 'Lácteos', 'nivel1': 'Alimentos'},
    'Yogurt': {'nivel2': 'Lácteos', 'nivel1': 'Alimentos'},
    'Pollo': {'nivel2': 'Carnes', 'nivel1': 'Alimentos'},
    'Carne_Vacuna': {'nivel2': 'Carnes', 'nivel1': 'Alimentos'},
    'Pan_Integral': {'nivel2': 'Panadería', 'nivel1': 'Alimentos'},
    'Pan_Blanco': {'nivel2': 'Panadería', 'nivel1': 'Alimentos'},
    'Laptop': {'nivel2': 'Computadoras', 'nivel1': 'Electrónicos'},
    'Mouse': {'nivel2': 'Accesorios', 'nivel1': 'Electrónicos'},
    'Teclado': {'nivel2': 'Accesorios', 'nivel1': 'Electrónicos'}
}

# Dataset de transacciones con productos específicos
transacciones_multilevel = [
    ['Leche_Entera', 'Pan_Integral', 'Pollo'],
    ['Leche_Descremada', 'Yogurt', 'Pan_Blanco'],
    ['Laptop', 'Mouse', 'Teclado'],
    ['Leche_Entera', 'Carne_Vacuna', 'Pan_Integral'],
    ['Yogurt', 'Pan_Blanco', 'Pollo'],
    ['Mouse', 'Teclado', 'Leche_Descremada'],
    ['Pan_Integral', 'Leche_Entera', 'Yogurt'],
    ['Laptop', 'Pan_Blanco'],
]

print("=== DATASET MULTILEVEL ===")
print("Transacciones originales (Nivel 3 - Específico):")
for i, trans in enumerate(transacciones_multilevel, 1):
    print(f"T{i}: {trans}")

def generar_nivel_superior(transacciones, nivel_destino):
    """Convierte transacciones a un nivel superior de la jerarquía"""
    transacciones_nivel = []
    for transaccion in transacciones:
        trans_nivel = []
        for item in transaccion:
            if item in jerarquia:
                item_nivel = jerarquia[item][nivel_destino]
                if item_nivel not in trans_nivel:  # Evitar duplicados
                    trans_nivel.append(item_nivel)
        transacciones_nivel.append(trans_nivel)
    return transacciones_nivel

# Generar transacciones en diferentes niveles
trans_nivel2 = generar_nivel_superior(transacciones_multilevel, 'nivel2')
trans_nivel1 = generar_nivel_superior(transacciones_multilevel, 'nivel1')

print("\nNivel 2 (Subcategorías):")
for i, trans in enumerate(trans_nivel2, 1):
    print(f"T{i}: {trans}")

print("\nNivel 1 (Categorías):")
for i, trans in enumerate(trans_nivel1, 1):
    print(f"T{i}: {trans}")


In [ ]:
# Funciones auxiliares del algoritmo Apriori (reutilizadas)
def calcular_soporte(itemset, transacciones):
    """Calcula el soporte de un itemset en las transacciones"""
    contador = 0
    for transaccion in transacciones:
        if set(itemset).issubset(set(transaccion)):
            contador += 1
    return contador / len(transacciones)

def apriori_simple(transacciones, soporte_min):
    """Versión simplificada de Apriori para reglas multilevel"""
    # Obtener items únicos
    items = set()
    for trans in transacciones:
        items.update(trans)
    
    # Encontrar itemsets frecuentes de tamaño 1
    itemsets_1 = []
    for item in items:
        if calcular_soporte([item], transacciones) >= soporte_min:
            itemsets_1.append([item])
    
    # Encontrar itemsets frecuentes de tamaño 2
    itemsets_2 = []
    for i in range(len(itemsets_1)):
        for j in range(i + 1, len(itemsets_1)):
            itemset = itemsets_1[i] + itemsets_1[j]
            if calcular_soporte(itemset, transacciones) >= soporte_min:
                itemsets_2.append(itemset)
    
    return itemsets_1 + itemsets_2

# Aplicar minería en cada nivel
print("\n=== REGLAS MULTILEVEL ===")

# Nivel 1 (Categorías generales)
print("1. Análisis en Nivel 1 (Categorías):")
itemsets_nivel1 = apriori_simple(trans_nivel1, 0.25)
print("Itemsets frecuentes:")
for itemset in itemsets_nivel1:
    soporte = calcular_soporte(itemset, trans_nivel1)
    print(f"  {itemset}: {soporte:.3f}")

# Nivel 2 (Subcategorías)
print("\n2. Análisis en Nivel 2 (Subcategorías):")
itemsets_nivel2 = apriori_simple(trans_nivel2, 0.25)
print("Itemsets frecuentes:")
for itemset in itemsets_nivel2:
    soporte = calcular_soporte(itemset, trans_nivel2)
    print(f"  {itemset}: {soporte:.3f}")

# Nivel 3 (Items específicos)
print("\n3. Análisis en Nivel 3 (Items específicos):")
itemsets_nivel3 = apriori_simple(transacciones_multilevel, 0.25)
print("Itemsets frecuentes:")
for itemset in itemsets_nivel3:
    soporte = calcular_soporte(itemset, transacciones_multilevel)
    print(f"  {itemset}: {soporte:.3f}")

# Ejemplo de refinamiento progresivo
print("\n=== REFINAMIENTO PROGRESIVO ===")
print("Regla general: Alimentos → frecuente en nivel 1")
print("Refinamiento: Lácteos → frecuente en nivel 2") 
print("Específico: Leche_Entera → frecuente en nivel 3")


In [ ]:
# Implementación de Reglas Generalizadas
def es_ancestro(item1, item2, jerarquia):
    """Verifica si item1 es ancestro de item2 en la jerarquía"""
    if item2 not in jerarquia:
        return False
    
    # Verificar en nivel 2
    if jerarquia[item2]['nivel2'] == item1:
        return True
    # Verificar en nivel 1
    if jerarquia[item2]['nivel1'] == item1:
        return True
    
    return False

def filtrar_reglas_taxonomicas(reglas_candidatas, jerarquia):
    """Filtra reglas que son obvias por la taxonomía"""
    reglas_filtradas = []
    
    for regla in reglas_candidatas:
        antecedente = regla['antecedente'] 
        consecuente = regla['consecuente']
        
        # Verificar si es una regla taxonómica obvia
        es_taxonomica = False
        for ant in antecedente:
            for cons in consecuente:
                if es_ancestro(cons, ant, jerarquia) or es_ancestro(ant, cons, jerarquia):
                    es_taxonomica = True
                    break
            if es_taxonomica:
                break
        
        if not es_taxonomica:
            reglas_filtradas.append(regla)
    
    return reglas_filtradas

def generar_reglas_generalizadas(transacciones_orig, jerarquia, soporte_min=0.25, confianza_min=0.5):
    """Genera reglas generalizadas combinando múltiples niveles"""
    print("=== REGLAS GENERALIZADAS ===")
    
    # Crear transacciones expandidas con múltiples niveles
    transacciones_expandidas = []
    for transaccion in transacciones_orig:
        trans_expandida = list(transaccion)  # Items originales
        
        # Agregar items de nivel superior
        for item in transaccion:
            if item in jerarquia:
                nivel2 = jerarquia[item]['nivel2']
                nivel1 = jerarquia[item]['nivel1']
                if nivel2 not in trans_expandida:
                    trans_expandida.append(nivel2)
                if nivel1 not in trans_expandida:
                    trans_expandida.append(nivel1)
        
        transacciones_expandidas.append(trans_expandida)
    
    print("Transacciones expandidas (con jerarquía):")
    for i, trans in enumerate(transacciones_expandidas[:3], 1):  # Mostrar solo 3
        print(f"T{i}: {trans}")
    print("...")
    
    # Encontrar itemsets frecuentes en transacciones expandidas
    itemsets_freq = apriori_simple(transacciones_expandidas, soporte_min)
    
    # Generar reglas candidatas
    reglas_candidatas = []
    for itemset in itemsets_freq:
        if len(itemset) >= 2:
            for i in range(1, len(itemset)):
                for antecedente in combinations(itemset, i):
                    antecedente = list(antecedente)
                    consecuente = [item for item in itemset if item not in antecedente]
                    
                    # Calcular métricas
                    soporte_union = calcular_soporte(itemset, transacciones_expandidas)
                    soporte_ant = calcular_soporte(antecedente, transacciones_expandidas)
                    
                    if soporte_ant > 0:
                        confianza = soporte_union / soporte_ant
                        
                        if confianza >= confianza_min:
                            reglas_candidatas.append({
                                'antecedente': antecedente,
                                'consecuente': consecuente,
                                'soporte': soporte_union,
                                'confianza': confianza
                            })
    
    # Filtrar reglas taxonómicas obvias
    reglas_filtradas = filtrar_reglas_taxonomicas(reglas_candidatas, jerarquia)
    
    print(f"\nReglas totales encontradas: {len(reglas_candidatas)}")
    print(f"Reglas después de filtrar taxonómicas: {len(reglas_filtradas)}")
    
    print("\nReglas generalizadas interesantes:")
    for i, regla in enumerate(reglas_filtradas[:5], 1):  # Mostrar top 5
        print(f"{i}. {regla['antecedente']} → {regla['consecuente']}")
        print(f"   Soporte: {regla['soporte']:.3f}, Confianza: {regla['confianza']:.3f}")
    
    return reglas_filtradas

# Ejecutar análisis de reglas generalizadas
reglas_generalizadas = generar_reglas_generalizadas(transacciones_multilevel, jerarquia)


In [ ]:
# Implementación de Reglas de Asociación Temporales
from datetime import datetime, timedelta
import calendar

# Dataset con timestamps (simulando compras a lo largo del año)
transacciones_temporales = [
    {'fecha': '2023-01-15', 'items': ['Abrigo', 'Bufanda', 'Guantes']},
    {'fecha': '2023-02-10', 'items': ['Chocolate', 'Flores', 'Tarjeta']},  # San Valentín
    {'fecha': '2023-03-20', 'items': ['Semillas', 'Fertilizante', 'Pala']},  # Primavera
    {'fecha': '2023-06-15', 'items': ['Protector_Solar', 'Sombrilla', 'Helado']},  # Verano
    {'fecha': '2023-07-04', 'items': ['Fuegos_Artificiales', 'Barbacoa', 'Bebidas']},
    {'fecha': '2023-10-31', 'items': ['Dulces', 'Decoración_Halloween', 'Disfraces']},
    {'fecha': '2023-11-20', 'items': ['Pavo', 'Salsa_Arándanos', 'Puré_Papa']},  # Thanksgiving
    {'fecha': '2023-12-15', 'items': ['Árbol_Navidad', 'Luces', 'Regalos']},
    {'fecha': '2023-12-20', 'items': ['Regalos', 'Papel_Regalo', 'Moños']},
    {'fecha': '2023-01-10', 'items': ['Abrigo', 'Guantes', 'Té_Caliente']},
    {'fecha': '2023-06-20', 'items': ['Protector_Solar', 'Gafas_Sol', 'Helado']},
    {'fecha': '2023-12-24', 'items': ['Regalos', 'Champagne', 'Árbol_Navidad']}
]

def extraer_estacion(fecha_str):
    """Extrae la estación del año de una fecha"""
    fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
    mes = fecha.month
    
    if mes in [12, 1, 2]:
        return 'Invierno'
    elif mes in [3, 4, 5]:
        return 'Primavera'
    elif mes in [6, 7, 8]:
        return 'Verano'
    else:
        return 'Otoño'

def extraer_mes(fecha_str):
    """Extrae el mes de una fecha"""
    fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
    return calendar.month_name[fecha.month]

# Análisis por estaciones
print("=== ANÁLISIS TEMPORAL POR ESTACIONES ===")

# Agrupar transacciones por estación
transacciones_por_estacion = {}
for trans in transacciones_temporales:
    estacion = extraer_estacion(trans['fecha'])
    if estacion not in transacciones_por_estacion:
        transacciones_por_estacion[estacion] = []
    transacciones_por_estacion[estacion].append(trans['items'])

# Mostrar patrones por estación
for estacion, transacciones in transacciones_por_estacion.items():
    print(f"\n{estacion}:")
    print(f"  Transacciones: {len(transacciones)}")
    
    # Encontrar items más frecuentes en esta estación
    frecuencia_items = {}
    for trans in transacciones:
        for item in trans:
            frecuencia_items[item] = frecuencia_items.get(item, 0) + 1
    
    # Mostrar top 3 items
    items_ordenados = sorted(frecuencia_items.items(), key=lambda x: x[1], reverse=True)
    print(f"  Items más frecuentes: {items_ordenados[:3]}")

def calcular_soporte_temporal(itemset, transacciones_fecha, ventana_temporal=None):
    """Calcula soporte considerando ventana temporal"""
    if ventana_temporal is None:
        # Sin restricción temporal
        return calcular_soporte(itemset, [t['items'] for t in transacciones_fecha])
    
    # Con restricción temporal (por estación/mes)
    transacciones_filtradas = []
    for trans in transacciones_fecha:
        if ventana_temporal == 'Invierno' and extraer_estacion(trans['fecha']) == 'Invierno':
            transacciones_filtradas.append(trans['items'])
        elif ventana_temporal == 'Verano' and extraer_estacion(trans['fecha']) == 'Verano':
            transacciones_filtradas.append(trans['items'])
        # ... más condiciones según necesidad
    
    if not transacciones_filtradas:
        return 0
    
    return calcular_soporte(itemset, transacciones_filtradas)

# Ejemplo de reglas temporales específicas
print("\n=== REGLAS TEMPORALES ESPECÍFICAS ===")

# Reglas de Invierno
reglas_invierno = [
    (['Abrigo'], ['Guantes']),
    (['Bufanda'], ['Abrigo']),
]

print("Reglas de Invierno:")
for antecedente, consecuente in reglas_invierno:
    soporte_invierno = calcular_soporte_temporal(
        antecedente + consecuente, transacciones_temporales, 'Invierno')
    soporte_general = calcular_soporte_temporal(
        antecedente + consecuente, transacciones_temporales, None)
    
    print(f"  {antecedente} → {consecuente}")
    print(f"    Soporte en Invierno: {soporte_invierno:.3f}")
    print(f"    Soporte general: {soporte_general:.3f}")
    print(f"    Factor estacional: {soporte_invierno/max(soporte_general, 0.001):.2f}")

# Análisis de ventanas deslizantes
print("\n=== VENTANAS DESLIZANTES (MESES) ===")

# Agrupar por meses
transacciones_por_mes = {}
for trans in transacciones_temporales:
    mes = extraer_mes(trans['fecha'])
    if mes not in transacciones_por_mes:
        transacciones_por_mes[mes] = []
    transacciones_por_mes[mes].append(trans['items'])

# Mostrar evolución temporal de items clave
items_seguimiento = ['Regalos', 'Protector_Solar', 'Abrigo']
print("Evolución temporal de items clave:")
for item in items_seguimiento:
    print(f"\n{item}:")
    for mes, transacciones in transacciones_por_mes.items():
        soporte = calcular_soporte([item], transacciones) if transacciones else 0
        print(f"  {mes}: {soporte:.2f}")


In [ ]:
# Implementación de Análisis de Secuencias
from collections import defaultdict

# Dataset de secuencias de compras por cliente
secuencias_compras = {
    'Cliente_1': [
        {'tiempo': 1, 'items': ['Pan', 'Leche']},
        {'tiempo': 2, 'items': ['Mantequilla']},
        {'tiempo': 4, 'items': ['Huevos', 'Queso']},
    ],
    'Cliente_2': [
        {'tiempo': 1, 'items': ['Pan']},
        {'tiempo': 3, 'items': ['Leche', 'Mantequilla']},
        {'tiempo': 5, 'items': ['Huevos']},
    ],
    'Cliente_3': [
        {'tiempo': 1, 'items': ['Leche']},
        {'tiempo': 2, 'items': ['Pan', 'Mantequilla']},
        {'tiempo': 3, 'items': ['Huevos']},
    ],
    'Cliente_4': [
        {'tiempo': 1, 'items': ['Pan', 'Queso']},
        {'tiempo': 3, 'items': ['Leche']},
        {'tiempo': 4, 'items': ['Mantequilla', 'Huevos']},
    ],
    'Cliente_5': [
        {'tiempo': 1, 'items': ['Mantequilla']},
        {'tiempo': 2, 'items': ['Pan', 'Leche']},
        {'tiempo': 4, 'items': ['Huevos']},
    ]
}

print("=== DATASET DE SECUENCIAS ===")
for cliente, secuencia in secuencias_compras.items():
    print(f"{cliente}: ", end="")
    secuencia_str = " → ".join([f"T{evento['tiempo']}:{evento['items']}" 
                               for evento in secuencia])
    print(secuencia_str)

def generar_subsecuencias_1(secuencias):
    """Genera todas las subsecuencias de tamaño 1"""
    items_unicos = set()
    for cliente, secuencia in secuencias.items():
        for evento in secuencia:
            items_unicos.update(evento['items'])
    
    return [[item] for item in items_unicos]

def contiene_subsecuencia(subsecuencia, secuencia_cliente):
    """Verifica si una subsecuencia está contenida en la secuencia de un cliente"""
    # Para subsecuencias simples de tamaño 1
    if len(subsecuencia) == 1:
        item_buscado = subsecuencia[0]
        for evento in secuencia_cliente:
            if item_buscado in evento['items']:
                return True
        return False
    
    # Para subsecuencias de tamaño 2
    if len(subsecuencia) == 2:
        item1, item2 = subsecuencia
        
        # Buscar item1 primero
        encontrado_item1 = False
        tiempo_item1 = 0
        
        for evento in secuencia_cliente:
            if item1 in evento['items']:
                encontrado_item1 = True
                tiempo_item1 = evento['tiempo']
                break
        
        if not encontrado_item1:
            return False
        
        # Buscar item2 después de item1
        for evento in secuencia_cliente:
            if evento['tiempo'] > tiempo_item1 and item2 in evento['items']:
                return True
        
        return False
    
    return False

def calcular_soporte_secuencial(subsecuencia, secuencias):
    """Calcula el soporte de una subsecuencia"""
    clientes_que_contienen = 0
    total_clientes = len(secuencias)
    
    for cliente, secuencia in secuencias.items():
        if contiene_subsecuencia(subsecuencia, secuencia):
            clientes_que_contienen += 1
    
    return clientes_que_contienen / total_clientes

def mineria_secuencias_simple(secuencias, soporte_min):
    """Algoritmo simple de minería de secuencias"""
    print(f"\n=== MINERÍA DE SECUENCIAS (soporte mín: {soporte_min}) ===")
    
    # Paso 1: Encontrar secuencias frecuentes de tamaño 1
    subsecuencias_1 = generar_subsecuencias_1(secuencias)
    secuencias_frecuentes_1 = []
    
    print("Secuencias frecuentes de longitud 1:")
    for subsec in subsecuencias_1:
        soporte = calcular_soporte_secuencial(subsec, secuencias)
        if soporte >= soporte_min:
            secuencias_frecuentes_1.append(subsec)
            print(f"  <{subsec[0]}>: {soporte:.3f}")
    
    # Paso 2: Generar candidatos de tamaño 2
    candidatos_2 = []
    for i in range(len(secuencias_frecuentes_1)):
        for j in range(len(secuencias_frecuentes_1)):
            if i != j:
                candidato = [secuencias_frecuentes_1[i][0], secuencias_frecuentes_1[j][0]]
                candidatos_2.append(candidato)
    
    # Encontrar secuencias frecuentes de tamaño 2
    secuencias_frecuentes_2 = []
    print("\nSecuencias frecuentes de longitud 2:")
    for candidato in candidatos_2:
        soporte = calcular_soporte_secuencial(candidato, secuencias)
        if soporte >= soporte_min:
            secuencias_frecuentes_2.append(candidato)
            print(f"  <{candidato[0]}, {candidato[1]}>: {soporte:.3f}")
    
    return secuencias_frecuentes_1, secuencias_frecuentes_2

# Ejecutar minería de secuencias
secuencias_freq_1, secuencias_freq_2 = mineria_secuencias_simple(secuencias_compras, 0.4)

# Análisis de patrones encontrados
print("\n=== ANÁLISIS DE PATRONES SECUENCIALES ===")
print("Interpretación de patrones frecuentes:")

for secuencia in secuencias_freq_2:
    print(f"\nPatrón: {secuencia[0]} → {secuencia[1]}")
    
    # Calcular estadísticas adicionales
    clientes_con_patron = []
    for cliente, sec_cliente in secuencias_compras.items():
        if contiene_subsecuencia(secuencia, sec_cliente):
            clientes_con_patron.append(cliente)
    
    print(f"  Clientes que siguen este patrón: {clientes_con_patron}")
    print(f"  Interpretación de negocio: Los clientes que compran {secuencia[0]} ")
    print(f"  tienden a comprar {secuencia[1]} en compras posteriores")


In [ ]:
# Implementación de Restricciones Temporales

def validar_restricciones_temporales(secuencia, restricciones):
    """Valida si una secuencia cumple las restricciones temporales"""
    
    # Restricción de gap máximo
    if 'gap_maximo' in restricciones:
        gap_max = restricciones['gap_maximo']
        for i in range(len(secuencia) - 1):
            gap = secuencia[i+1]['tiempo'] - secuencia[i]['tiempo']
            if gap > gap_max:
                return False
    
    # Restricción de gap mínimo
    if 'gap_minimo' in restricciones:
        gap_min = restricciones['gap_minimo']
        for i in range(len(secuencia) - 1):
            gap = secuencia[i+1]['tiempo'] - secuencia[i]['tiempo']
            if gap < gap_min:
                return False
    
    # Restricción de duración total
    if 'duracion_maxima' in restricciones:
        duracion_max = restricciones['duracion_maxima']
        duracion_total = secuencia[-1]['tiempo'] - secuencia[0]['tiempo']
        if duracion_total > duracion_max:
            return False
    
    # Restricción de longitud
    if 'longitud_minima' in restricciones:
        if len(secuencia) < restricciones['longitud_minima']:
            return False
    
    return True

def mineria_secuencias_con_restricciones(secuencias, soporte_min, restricciones):
    """Minería de secuencias con restricciones temporales"""
    print(f"\n=== MINERÍA CON RESTRICCIONES TEMPORALES ===")
    print(f"Restricciones aplicadas: {restricciones}")
    
    # Filtrar secuencias que cumplen restricciones
    secuencias_filtradas = {}
    for cliente, secuencia in secuencias.items():
        if validar_restricciones_temporales(secuencia, restricciones):
            secuencias_filtradas[cliente] = secuencia
    
    print(f"Secuencias que cumplen restricciones: {len(secuencias_filtradas)}/{len(secuencias)}")
    
    # Aplicar minería sobre secuencias filtradas
    if secuencias_filtradas:
        return mineria_secuencias_simple(secuencias_filtradas, soporte_min)
    else:
        print("No hay secuencias que cumplan las restricciones")
        return [], []

# Ejemplo 1: Restricción de gap máximo
print("=== EJEMPLO 1: GAP MÁXIMO ===")
restricciones_gap = {'gap_maximo': 2}
secuencias_gap_1, secuencias_gap_2 = mineria_secuencias_con_restricciones(
    secuencias_compras, 0.4, restricciones_gap)

# Ejemplo 2: Restricción de duración total
print("\n=== EJEMPLO 2: DURACIÓN TOTAL ===")
restricciones_duracion = {'duracion_maxima': 3}
secuencias_dur_1, secuencias_dur_2 = mineria_secuencias_con_restricciones(
    secuencias_compras, 0.4, restricciones_duracion)

# Ejemplo 3: Múltiples restricciones
print("\n=== EJEMPLO 3: MÚLTIPLES RESTRICCIONES ===")
restricciones_multiples = {
    'gap_maximo': 3,
    'longitud_minima': 3,
    'duracion_maxima': 4
}
secuencias_mult_1, secuencias_mult_2 = mineria_secuencias_con_restricciones(
    secuencias_compras, 0.4, restricciones_multiples)

# Análisis de secuencias jerárquicas
def generar_secuencias_jerarquicas(secuencias_orig, jerarquia):
    """Convierte secuencias específicas a secuencias jerárquicas"""
    secuencias_jerarquicas = {}
    
    for cliente, secuencia in secuencias_orig.items():
        secuencia_jerarquica = []
        
        for evento in secuencia:
            items_jerarquicos = []
            for item in evento['items']:
                if item in jerarquia:
                    items_jerarquicos.append(jerarquia[item]['nivel2'])
            
            if items_jerarquicos:
                secuencia_jerarquica.append({
                    'tiempo': evento['tiempo'],
                    'items': list(set(items_jerarquicos))  # Eliminar duplicados
                })
        
        secuencias_jerarquicas[cliente] = secuencia_jerarquica
    
    return secuencias_jerarquicas

print("\n=== ANÁLISIS JERÁRQUICO DE SECUENCIAS ===")

# Mapeo de items a jerarquía para el dataset de secuencias
jerarquia_secuencias = {
    'Pan': {'nivel2': 'Panadería', 'nivel1': 'Alimentos'},
    'Leche': {'nivel2': 'Lácteos', 'nivel1': 'Alimentos'},
    'Mantequilla': {'nivel2': 'Lácteos', 'nivel1': 'Alimentos'},
    'Huevos': {'nivel2': 'Carnes', 'nivel1': 'Alimentos'},
    'Queso': {'nivel2': 'Lácteos', 'nivel1': 'Alimentos'},
}

secuencias_jerarq = generar_secuencias_jerarquicas(secuencias_compras, jerarquia_secuencias)

print("Secuencias jerárquicas (nivel subcategoría):")
for cliente, secuencia in secuencias_jerarq.items():
    print(f"{cliente}: ", end="")
    secuencia_str = " → ".join([f"T{evento['tiempo']}:{evento['items']}" 
                               for evento in secuencia])
    print(secuencia_str)

# Minería en secuencias jerárquicas
secuencias_jerarq_freq = mineria_secuencias_simple(secuencias_jerarq, 0.4)
